# Effect of expertise, nativity, and fluency

- group comparisons using % agreement, Spearman r, Krippendorf alpha, Fleiss kapp etc.

In [ ]:
# intro, defaults (shared across notebooks)

import pandas as pd
import numpy as np
import scipy.stats as st
from pandas import DataFrame, Series, MultiIndex
import matplotlib.pyplot as plt
import drawBot as db
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pingouin as pg
import krippendorff
%matplotlib inline


# ---------------------------------------------------
# helpers

def fix_columns(cols):
    for col in cols:
        if col[0] == "control":
            yield col
        else:
            col = list(col)
            col[-1] = tuple(eval(col[-1]))
            yield tuple(col)

def make_compact(d, flat=False):
    """
    Make DataFrame compact = use [0, 1, 2]
    instead of all characters on the index
    """

    d_compact = pd.DataFrame(columns=d.columns, index=[0, 1, 2])
    for col in d.columns:
        triplet = col[-1] # last item in the index is a triplet
        col_data = pd.Series(d[col].dropna(), index=triplet)
        d_compact[col] = list(col_data) # past regardless the index
    if flat:
        return d_compact.fillna(0)
    else:
        return d_compact.fillna(0)


# ---------------------------------------------------
# set global properties for plotting

font = {"family":"Adapter Mono PE", "size":"10", "weight":"medium"}
figure = {"titlesize":"10","titleweight":"medium"}
axes = {"titlesize":"10", "titleweight":"medium", "labelsize":"10", "labelweight":"medium"}
plt.rc("font", **font)
plt.rc("figure", **figure)
plt.rc("axes", **axes)
box_colors = dict(boxes="Black", whiskers="0.5", medians="Black", caps="0.5")

In [ ]:
# Get data

d = pd.read_csv("csv/raw-data-preprocessed.csv", index_col=0, header=[0,1,2], dtype="unicode")
d.sort_index(axis=1, inplace=True)

non_control_columns = [col for col in d.columns if col[0] != "control"]

# fix type
d["control", "control", "order"] = d["control", "control", "order"].astype("float").astype("int")

# fix triplet columns (convert them to tuples)
d.columns = pd.MultiIndex.from_tuples(list(fix_columns(d.columns)))

# data frame just for the demographics
demo = d["control", "control"]

print("Imported %d rows, %d columns." % d.shape)


# ---------------------------------------------------
# lists of typefaces and scripts

from chardict import chardict, all_characters, all_typefaces, all_script_typefaces

sorted_scripts = sorted(all_script_typefaces.keys())
# get sorting for all characters in all scripts together
sorted_chars = []
for script in ["cyrillic", "devanagari", "latin"]:
    sorted_chars += list(chardict[script].keys())

Imported 1787 rows, 1579 columns.


In [ ]:
# Frequencies & distribution of the most popular response (copied from analysis)

# functions

def get_frequencies_typeface(d, index=None, mode="relative", sort=False):
    """
    Find relative frequencies for characters in each triplet in DF.
    
    Parameters:
        index: is used to pass a shared index across multiple typefaces
        mode: indicates whether the counts should be normalized (i.e. relative frequencies)
        sort: whether to return a sorted Series instead of matching responses to characters
    Return:
        DF with frequencies.
    """
    
    # the index need to be built first
    # so value_counts below fall to the right slot
    if sort:
        index = [0, 1, 2]
    elif index is None:
        index = []
        for triplet in d.columns:
            index += list(triplet)
        index = sorted(list(set(index)))
    # count frequencies
    frequencies = DataFrame(index=index, columns=d.columns)
    if mode == "relative":
        for triplet in d.columns:
            # get relative frequencies for each character in the triplet
            if sort:
                value_counts = d[triplet].value_counts(dropna=True, normalize=True, ascending=True)
                value_counts.index = index[:len(value_counts)]
            else:
                value_counts = d[triplet].value_counts(dropna=True, normalize=True)
            if not value_counts.empty:
                frequencies[triplet] = value_counts
    elif mode == "absolute":
        for triplet in d.columns:
            # get frequencies for each character in the triplet, do not normalize
            if sort:
                value_counts = d[triplet].value_counts(dropna=True, ascending=True)
                value_counts.index = index[:len(value_counts)]
            else:
                value_counts = d[triplet].value_counts(dropna=True)
            if not value_counts.empty:
                frequencies[triplet] = value_counts
        
    return frequencies

def get_frequencies_script(d, mode="relative", sort=False):
    """
    Find relative frequencies for each script in DF.
    
    Parameters:
        mode: indicates whether the counts should be normalized (i.e. relative frequencies)
    Return:
        DF with frequencies.
    """
    
    # the index need to be built first
    # so value_counts below fall to the right slot
    if sort:
        index = [0, 1, 2]
    else:
        index = []
        for triplet in d.columns.levels[1]:
            index += list(triplet)
        index = sorted(list(set(index)))
    frequencies = DataFrame(index=index, columns=d.columns)
    for typeface in d.columns.levels[0]:
        frequencies[typeface] = get_frequencies_typeface(d[typeface], index=index, mode=mode, sort=sort)
        
    return frequencies

def get_frequencies(d_, mode="relative", sort=False):
    """
    Get relative frequencies for all scripts, everything.
    
    Parameters:
        mode: indicates whether the counts should be normalized (i.e. relative frequencies)
    """
    
    d = d_.loc[:,d_.columns.get_level_values(0).isin(["cyrillic", "devanagari", "latin"])]
    d.columns = d.columns.remove_unused_levels()
    d.sort_index(axis=1, inplace=True)
    # in case the DF contains multiple scripts, flatten it
    if len(d.columns.levels) > 2:
        backup_cols = d.columns
        dt = d
        dt.columns = dt.columns.droplevel(level=0).unique()
        frequencies = get_frequencies_script(dt, mode=mode, sort=sort)
        frequencies.columns = backup_cols
    else:
        frequencies = get_frequencies_script(d, mode=mode, sort=sort)
    return frequencies


def get_majority(counts):
    """
    Convert probabilities to classification based on majority vote.
    """
    
    majority = pd.DataFrame(0, columns=counts.columns, index=counts.index)
    for col in counts:
        i = counts[col].idxmax()
        majority[col][i] = 1
    return majority

In [ ]:
# Agreement across different groups

def format_condition(column, value, script=None, ignore_second=True):
    """
    Compile a condition to filter out dataset based on particular condition/value.
    """
    
    if value == "*":
        # show everything
        condition = True
    elif value == "Designer":
        # special case in design skills
        condition = (d["control", "control", column] != "Non-designer") & (d["control", "control", column] != "Letter designer")
    else:
        condition = (d[("control", "control", column)] == value)
    # filter script
    if script is not None:
        condition &= (d[("control", "control", "script")] == script)
    # ignore the second session from two-typeface variant of the experiments
    if ignore_second:
        condition &= (d[("control", "control", "order")] == 1)
    
    return condition

def get_shared_cols(*data, ignore_controls=True):
    """
    Get a list of shared columns, optionally ignore control columns.
    """

    if ignore_controls:
        shared_cols = [c for c in data[0].columns if c[0] != "control"]
    else:
        shared_cols = data[0].columns
    for df in data:
        shared_cols = [c for c in shared_cols if c in df]
    return shared_cols

def compare_data(data, fast=False):
    """
    Compare two data sets and return statistics:
    - simple agreement on the OOOO response (0.0–1.0)
    - Spearman’s rank correlation (Pearson is not a good fit, non-normal distribution [although close])
    - mean-squared error (MSE)
    # - R2 coef.
    # - Fleiss’s kappa
    # - ANOVA of means? # test for bivariate normality? # display(pg.multivariate_normality())
    """

    # get frequencies, responses, oooo_frquencies for these trials
    # and compare
    compact_frqs = []
    oooo_frqs = []

    # get shared trials
    MIN_PARTICIPANTS = 5  # minimal number of participants in a subsetted trial data
    MIN_TRIALS = 10  # minimal number of trials in data sets
    shared_cols = get_shared_cols(*data)
    # use any trials with at least MIN_PARTICIPANTS of participants
    for part in data:
        shared_cols = [col for col in shared_cols if part[col].count() > MIN_PARTICIPANTS]
    # compare only if there are at least MIN_TRIALS
    if len(shared_cols) < MIN_TRIALS:
        return None

    for part in data:
        # get all response frequencies in canonical order
        # and make them compact 1D array to use with correlation
        x = pd.DataFrame(part, columns=shared_cols)
        frqs = get_frequencies(x, mode="relative").reindex(sorted_chars)
        if not fast:
            compact_frqs.append(make_compact(frqs).values.flatten(order="F"))
        # get the OOOO response frequencies     
        oooo_frqs.append([frqs[col].idxmax(skipna=True) for col in frqs])

    if fast:
        agreement = pd.Series([v1 == v2 for v1, v2 in zip(*oooo_frqs)])
        #correlation = pg.corr(*compact_frqs, method="spearman", alternative="two-sided")
        #mse = mean_squared_error(*compact_frqs)
        return pd.Series({
            "trials": len(shared_cols),
            "agreement mean": agreement.mean(),
            #"agreement sem": agreement.sem(),
            #"spearman r": correlation.loc["spearman", "r"].round(3),
            #"spearman p-val": correlation.loc["spearman", "p-val"].round(3),
            #"mse": mse,
        })
    else:
        agreement = pd.Series([v1 == v2 for v1, v2 in zip(*oooo_frqs)])
        correlation = pg.corr(*compact_frqs, method="spearman", alternative="two-sided")
        mse = mean_squared_error(*compact_frqs)
        return pd.Series({
            "trials": len(shared_cols),
            "agreement mean": agreement.mean(),
            "agreement sem": agreement.sem(),
            "spearman r": correlation.loc["spearman", "r"].round(3),
            "spearman p-val": correlation.loc["spearman", "p-val"].round(3),
            "mse": mse,
        })

def format_results(stats, script, label):
    """
    Format resulting statistics into a DataFrame
    """

    COLUMNS = [
        # Friendly name, stats key or default value, True if stats key
        ("Script", "", False),
        ("Description", "", False),
        ("# participants", "participants", True),
        ("# trials", "trials", True),
        ("Agreement", "agreement mean", True),
        ("Spearman (r)", "spearman r", True),
        ("Spearman (p-val)", "spearman p-val", True),
        ("MSE", "mse", True),
        ("Krippendorff", "krippendorff", True),
        ("Fleiss", "fleiss", True),
    ]

    row = pd.DataFrame([], columns=[c[0] for c in COLUMNS], index=[0])
    row["Script"] = script.title()
    row["Description"] = label
    for k, sk, b in COLUMNS:
        if sk in stats:
            if k == "Agreement":
                row[k] = "%.2f %%" % (100 * stats[sk])
            elif k.startswith("#") and isinstance(stats[sk], (int, float)):
                row[k] = int(stats[sk])
            elif isinstance(stats[sk], str):
                row[k] = stats[sk]
            else:
                row[k] = round(stats[sk], 3)
        elif b:
            row[k] = "NA"
    return row


In [ ]:
from statsmodels.stats import inter_rater as irr

def compare_internal(data, fast=True, iter=1):
    """
    Get inter-rater comparison by halving the data
    and comparing the subsets to each other a few times
    """

    stats = []
    for _ in range(iter):
        # split in two halfs
        h1 = data.sample(frac=0.5).dropna(axis="columns", how="all")
        h2 = data.drop(h1.index).dropna(axis="columns", how="all")
        stats_ = compare_data([h1, h2], fast=fast)
        if stats_ is not None:
            stats.append(stats_)
    else:
        stats_ = pd.Series()
        cols = [c for c in data.columns if c[0] != "control"]
        data_ = pd.DataFrame(data, columns=cols)
        for c in cols:
            c_ = eval(c[2])
            data_[c] = [np.nan if pd.isnull(v) else c_.index(v) for v in data_[c].values]
        data_ = data_.astype(np.float32)

        stats_["trials"] = len(cols)
        stats_["krippendorff"] = krippendorff.alpha(reliability_data=data_, level_of_measurement="nominal")
        #stats_["fleiss"] = irr.fleiss_kappa(irr.aggregate_raters(data_)[0].transpose(), method='fleiss')
        stats.append(stats_)
    if stats in [[], None]:
        print("!! the small number of participants causes an insufficient overlap when splitting (?)")
        return None
    return pd.concat(stats, axis=1).aggregate("mean", axis=1)


def internal_comparison_report(d, script, description="All", column=None, value=None):
    """
    Report inter-rater comparison within the whole data set
    or a subset of the data set defined by a control column and its value
    """
    
    condition = format_condition(column, value, script)
    dt = d[condition].dropna(axis="columns", how="all")

    stats = compare_internal(dt, fast=False, iter=2)
    
    if stats is not None:
        # format results
        stats["participants"] = len(dt.index)
        row = format_results(stats, script, description)
        print(f"> {description} ({script}) calculated and added to the report")
        return row
    else:
        # format results
        row = format_results({"participants": len(dt.index)}, script, description)  # this is a number of participants??
        # an insufficient number of trials (<{MIN_TRIALS}) with >{MIN_PARTICIPANTS} participants
        print(f"! {description} ({script}) an insufficient number of trials or participants per trial in the selection")
        return row


# # Report inter-rater comparisons
for script in sorted_scripts:
    results = [
        internal_comparison_report(d, script, "All", value="*"),

        internal_comparison_report(d, script, "Non-designers", "design skills", "Non-designer"),
        internal_comparison_report(d, script, "Designers", "design skills", "Designer"),
        internal_comparison_report(d, script, "Letter designers", "design skills", "Letter designer"),

        internal_comparison_report(d, script, "Native", "native in script", "True"),
        internal_comparison_report(d, script, "Non-native", "native in script", "False"),

        internal_comparison_report(d, script, "Fluent", "fluent in script", "True"),
        internal_comparison_report(d, script, "Non-fluent", "fluent in script", "False"),
    ]
    display(pd.concat(results))



> All (cyrillic) calculated and added to the report
> Non-designers (cyrillic) calculated and added to the report
> Designers (cyrillic) calculated and added to the report
> Letter designers (cyrillic) calculated and added to the report
> Native (cyrillic) calculated and added to the report
> Non-native (cyrillic) calculated and added to the report
> Fluent (cyrillic) calculated and added to the report
> Non-fluent (cyrillic) calculated and added to the report


,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff,Fleiss
0,Cyrillic,All,509,448,90.29 %,0.908,0.0,0.01,0.354,NA
0,Cyrillic,Non-designers,361,448,88.62 %,0.875,0.0,0.014,0.348,NA
0,Cyrillic,Designers,121,280,80.58 %,0.813,0.0,0.027,0.379,NA
0,Cyrillic,Letter designers,27,448,NA,NA,NA,NA,0.334,NA
0,Cyrillic,Native,472,448,90.51 %,0.904,0.0,0.01,0.350,NA
0,Cyrillic,Non-native,37,448,NA,NA,NA,NA,0.430,NA
0,Cyrillic,Fluent,482,448,89.51 %,0.899,0.0,0.01,0.352,NA
0,Cyrillic,Non-fluent,27,448,NA,NA,NA,NA,0.425,NA


> All (devanagari) calculated and added to the report
> Non-designers (devanagari) calculated and added to the report
> Designers (devanagari) calculated and added to the report
> Letter designers (devanagari) calculated and added to the report
> Native (devanagari) calculated and added to the report
> Non-native (devanagari) calculated and added to the report
> Fluent (devanagari) calculated and added to the report
> Non-fluent (devanagari) calculated and added to the report


,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff,Fleiss
0,Devanagari,All,432,448,81.81 %,0.854,0.0,0.014,0.229,NA
0,Devanagari,Non-designers,345,448,80.92 %,0.829,0.0,0.017,0.231,NA
0,Devanagari,Designers,69,448,NA,NA,NA,NA,0.200,NA
0,Devanagari,Letter designers,18,336,NA,NA,NA,NA,0.444,NA
0,Devanagari,Native,259,448,78.46 %,0.79,0.0,0.021,0.221,NA
0,Devanagari,Non-native,173,392,78.95 %,0.767,0.0,0.029,0.256,NA
0,Devanagari,Fluent,356,448,82.37 %,0.834,0.0,0.016,0.224,NA
0,Devanagari,Non-fluent,76,186,67.86 %,0.584,0.0,0.061,0.283,NA


> All (latin) calculated and added to the report
> Non-designers (latin) calculated and added to the report
> Designers (latin) calculated and added to the report
> Letter designers (latin) calculated and added to the report
> Native (latin) calculated and added to the report
> Non-native (latin) calculated and added to the report
> Fluent (latin) calculated and added to the report


ValueError: There has to be at least one unit with values assigned by at least two coders.

In [27]:
def mutual_comparison_report(d, script, description, column, values):
    """
    Take two subsets from the data defined by
    control columns and their values and compare them
    """


    if len(values) != 2:
        print("Error.Need precisely two groups to compare")
        return
    
    # split based on condition
    dt = []
    for value in values:
        condition = format_condition(column, value, script)
        dtt = d[condition].dropna(axis="columns", how="all")
        dt.append(dtt)
    
    stats = compare_data(dt)
    
    if stats is not None:
        # format results
        stats["participants"] = "%d/%d" % (len(dt[0].index), len(dt[1].index))
        row = format_results(stats, script, description)
        print(f"+ {description} ({script}) calculated and added to the report")
        return row
    else:
        row = format_results({"participants": "%d/%d" % (len(dt[0].index), len(dt[1].index))}, script, description)
        # an insufficient number of trials (<{MIN_TRIALS}) with >{MIN_PARTICIPANTS} participants
        print(f"! {description} ({script}) an insufficient number of trials or participants per trial in either group")
        return row

# Report comparison between groups

for script in sorted_scripts:
    results = [
        mutual_comparison_report(d, script, "Non-designers vs. Designers", "design skills", ["Non-designer", "Designer"]),
        mutual_comparison_report(d, script, "Non-designers vs. Letter designers", "design skills", ["Non-designer", "Letter designer"]),
        mutual_comparison_report(d, script, "Designers vs. Letter designers", "design skills", ["Designer", "Letter designer"]),
    ]
    display(pd.concat(results))

for script in sorted_scripts:
    results = [
        mutual_comparison_report(d, script, "Native vs. Non-native participants", "native in script", ["True", "False"]),
        mutual_comparison_report(d, script, "Fluent vs. Non-fluent participants", "fluent in script", ["True", "False"]),
    ]
    display(pd.concat(results))

+ Non-designers vs. Designers (cyrillic) calculated and added to the report
! Non-designers vs. Letter designers (cyrillic) an insufficient number of trials or participants per trial in either group
! Designers vs. Letter designers (cyrillic) an insufficient number of trials or participants per trial in either group


,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff
0,Cyrillic,Non-designers vs. Designers,361/121,392,87.50 %,0.902,0.0,0.012,NA
0,Cyrillic,Non-designers vs. Letter designers,361/27,NA,NA,NA,NA,NA,NA
0,Cyrillic,Designers vs. Letter designers,121/27,NA,NA,NA,NA,NA,NA


+ Non-designers vs. Designers (devanagari) calculated and added to the report
! Non-designers vs. Letter designers (devanagari) an insufficient number of trials or participants per trial in either group
! Designers vs. Letter designers (devanagari) an insufficient number of trials or participants per trial in either group


,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff
0,Devanagari,Non-designers vs. Designers,345/69,392,75.26 %,0.744,0.0,0.028,NA
0,Devanagari,Non-designers vs. Letter designers,345/18,NA,NA,NA,NA,NA,NA
0,Devanagari,Designers vs. Letter designers,69/18,NA,NA,NA,NA,NA,NA


+ Non-designers vs. Designers (latin) calculated and added to the report
+ Non-designers vs. Letter designers (latin) calculated and added to the report
+ Designers vs. Letter designers (latin) calculated and added to the report


...

,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff
0,Cyrillic,Native vs. Non-native participants,472/37,112,89.29 %,0.868,0.0,0.018,NA
0,Cyrillic,Fluent vs. Non-fluent participants,482/27,56,80.36 %,0.901,0.0,0.016,NA


+ Native vs. Non-native participants (devanagari) calculated and added to the report
+ Fluent vs. Non-fluent participants (devanagari) calculated and added to the report


,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff
0,Devanagari,Native vs. Non-native participants,259/173,448,81.03 %,0.823,0.0,0.019,NA
0,Devanagari,Fluent vs. Non-fluent participants,356/76,448,78.35 %,0.780,0.0,0.029,NA


+ Native vs. Non-native participants (latin) calculated and added to the report
! Fluent vs. Non-fluent participants (latin) an insufficient number of trials or participants per trial in either group


,Script,Description,# participants,# trials,Agreement,Spearman (r),Spearman (p-val),MSE,Krippendorff
0,Latin,Native vs. Non-native participants,744/36,56,94.64 %,0.931,0.0,0.01,NA
0,Latin,Fluent vs. Non-fluent participants,778/2,NA,NA,NA,NA,NA,NA
